# Project Overview :
goal of our  project is to analyze high-paying jobs (those earning $100K or more) using U.S. Bureau of Labor Statistics (BLS) data, and relating this to educational attainment, occupation, gender, age, and income from a second dataset (Educational Attainment).
specifically, we  are:

- Working with the BLS data (which includes details like occupation codes, mean and median wages, etc.). as first dataset
- Using the Educational Attainment data to analyze how factors such as age, gender, and educational background influence high-paying jobs.as  second data 

we want  to merge these datasets (with a focus on region, occupation codes, and key demographic factors) to understand the relationship between these factors and high-paying occupations.

load libraries

In [ ]:
import pandas as pd
import numpy as np

## 1. Data processing : 

- Select Relevant Columns - Only keep the important columns for your analysis.
- Handle Missing Values - Either drop or fill missing data.
- Remove Duplicates - Remove duplicate rows from the dataset.
- Validate Data Types - Ensure numeric columns are properly formatted.
- Review Data Consistency - Check for consistency in categorical columns.

In [54]:
# Load the data using pandas
# bls_data=pd.read_excel("./Ressources/all_data_M_2023.xlsx")
# Preview the first few rows to check if the data is being read correctly
display(bls_data.head(3))
display(bls_data.tail(3))


,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,23.11,37.01,58.4,29050,35660,48060,76980,121470,NaN,NaN
1,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,56.19,81.29,111.36,54550,78330,116880,169090,231620,NaN,NaN
2,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,11-1000,Top Executives,...,49.74,79.57,#,46400,66170,103460,165500,#,NaN,NaN


,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
413324,5500002,Northeastern Wisconsin nonmetropolitan area,6,WI,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",...,14.11,18.29,26.99,20290,25080,29350,38050,56130,NaN,NaN
413325,5500003,South Central Wisconsin nonmetropolitan area,6,WI,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",...,15.94,18.55,20.54,28380,32680,33150,38590,42730,NaN,NaN
413326,5500004,Western Wisconsin nonmetropolitan area,6,WI,000000,Cross-industry,cross-industry,1235,53-7199,"Material Moving Workers, All Other",...,16.42,26.99,29.69,22130,28790,34160,56130,61740,NaN,NaN


In [28]:
# check the columns names
bls_data.columns

Index(['AREA', 'AREA_TITLE', 'AREA_TYPE', 'PRIM_STATE', 'NAICS', 'NAICS_TITLE',
       'I_GROUP', 'OWN_CODE', 'OCC_CODE', 'OCC_TITLE', 'O_GROUP', 'TOT_EMP',
       'EMP_PRSE', 'JOBS_1000', 'LOC_QUOTIENT', 'PCT_TOTAL', 'PCT_RPT',
       'H_MEAN', 'A_MEAN', 'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN',
       'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75',
       'A_PCT90', 'ANNUAL', 'HOURLY'],
      dtype='object')

In our  BLS data, the most relevant columns to keep depend on our project's focus: 

- AREA and AREA_TITLE: To identify the geographic region and perform location-based analysis.
- NAICS and NAICS_TITLE: For industry classification, which helps analyze trends by industry sector.
- OCC_CODE and OCC_TITLE: For occupation-specific analysis, which is crucial when looking at job types and wages.
- TOT_EMP: Total employment helps in analyzing job concentration and demand.
- H_MEAN and A_MEAN: Hourly and annual mean wages are essential for identifying high-paying jobs.
- H_MEDIAN and A_MEDIAN: Median wages to assess typical earnings in each role.
- H_PCT75 and A_PCT75, H_PCT90 and A_PCT90: These percentile columns help in understanding wage distribution at higher levels, useful for identifying the top earners.

## SELECT THE RELEVANT COLUMNS

 >the columns to keep:
* OCC_CODE and OCC_TITLE: For analyzing specific occupations and job categories.
* NAICS and NAICS_TITLE: If you're looking to correlate job salaries with specific industries.
* AREA and AREA_TITLE: If you're interested in geographic variations, such as comparing salaries across regions.

In [56]:
# Select the relevant columns from your DataFrame
relevant_columns = [
    'AREA', 'AREA_TITLE', 'NAICS_TITLE', 
    'OCC_CODE', 'OCC_TITLE', 'TOT_EMP', 'PRIM_STATE',
    'H_MEAN', 'A_MEAN', 'H_MEDIAN', 'A_MEDIAN',
    'H_PCT75', 'A_PCT75', 'H_PCT90', 'A_PCT90'
]

# Subset the DataFrame
bls_df = bls_data[relevant_columns]
bls_df.head()

,AREA,AREA_TITLE,NAICS_TITLE,OCC_CODE,OCC_TITLE,TOT_EMP,PRIM_STATE,H_MEAN,A_MEAN,H_MEDIAN,A_MEDIAN,H_PCT75,A_PCT75,H_PCT90,A_PCT90
0,99,U.S.,Cross-industry,00-0000,All Occupations,151853870,US,31.48,65470,23.11,48060,37.01,76980,58.4,121470
1,99,U.S.,Cross-industry,11-0000,Management Occupations,10495770,US,66.23,137750,56.19,116880,81.29,169090,111.36,231620
2,99,U.S.,Cross-industry,11-1000,Top Executives,3751510,US,65.43,136100,49.74,103460,79.57,165500,#,#
3,99,U.S.,Cross-industry,11-1010,Chief Executives,211230,US,124.47,258900,99.37,206680,#,#,#,#
4,99,U.S.,Cross-industry,11-1011,Chief Executives,211230,US,124.47,258900,99.37,206680,#,#,#,#


In [ ]:
# Remove hyphen from the OCC_CODE column
bls_df['OCC_CODE'] = bls_df['OCC_CODE'].str.replace('-', '', regex=False)

C:\Users\marwa\AppData\Local\Temp\ipykernel_10028\452566841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bls_df['OCC_CODE'] = bls_df['OCC_CODE'].str.replace('-', '', regex=False)


In [60]:
# Get the unique values for each column in the DataFrame
unique_values = bls_df.apply(lambda x: x.unique())

# Display the unique values for each column
print(unique_values)

AREA           [99, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, ...
AREA_TITLE     [U.S., Alabama, Alaska, Arizona, Arkansas, Cal...
NAICS_TITLE    [Cross-industry, Cross-industry, Private Owner...
OCC_CODE       [000000, 110000, 111000, 111010, 111011, 11102...
OCC_TITLE      [All Occupations, Management Occupations, Top ...
TOT_EMP        [151853870, 10495770, 3751510, 211230, 3507810...
PRIM_STATE     [US, AL, AK, AZ, AR, CA, CO, CT, DE, DC, FL, G...
H_MEAN         [31.48, 66.23, 65.43, 124.47, 62.18, *, 76.9, ...
A_MEAN         [65470, 137750, 136100, 258900, 129330, 68140,...
H_MEDIAN       [23.11, 56.19, 49.74, 99.37, 48.69, *, 67.23, ...
A_MEDIAN       [48060, 116880, 103460, 206680, 101280, 47290,...
H_PCT75        [37.01, 81.29, 79.57, #, 77.06, *, 96.15, 90.6...
A_PCT75        [76980, 169090, 165500, #, 160290, 82200, 1999...
H_PCT90        [58.4, 111.36, #, 111.59, *, 102.27, 86.41, 90...
A_PCT90        [121470, 231620, #, 232110, 129510, 212730, 17...
dtype: object


In [61]:
# Replace multiple specific special characters with NaN
bls_df.replace({'#': np.nan, '*': np.nan, '@': np.nan, '$': np.nan}, inplace=True)

C:\Users\marwa\AppData\Local\Temp\ipykernel_10028\3196695766.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bls_df.replace({'#': np.nan, '*': np.nan, '@': np.nan, '$': np.nan}, inplace=True)
C:\Users\marwa\AppData\Local\Temp\ipykernel_10028\3196695766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bls_df.replace({'#': np.nan, '*': np.nan, '@': np.nan, '$': np.nan}, inplace=True)


### Handel missing values

In [32]:
# Check for missing values in the selected columns
missing_values = bls_df.isnull().sum()
print(missing_values)

AREA               0
AREA_TITLE         0
NAICS              0
NAICS_TITLE        0
OCC_CODE           0
OCC_TITLE          0
TOT_EMP            0
PRIM_STATE         0
H_MEAN         20964
A_MEAN          5352
H_MEDIAN       23059
A_MEDIAN        7474
H_PCT75        26119
A_PCT75        10606
H_PCT90        33008
A_PCT90        17844
dtype: int64


In [62]:

# drop the nmissing values 
bls_df= bls_df.dropna(how='any')
# check if we do have some duplicated valuesto be dorped 
# bls_df_clean.duplicated()

In [34]:
# Remove duplicate rows if any
bls_df_=bls_df.drop_duplicates(inplace=True)

In [40]:
# chek the how many classified occ we do have 
bls_df['OCC_TITLE'].value_counts()


OCC_TITLE
Office Clerks, General                                                 1470
First-Line Supervisors of Office and Administrative Support Workers    1464
Bookkeeping, Accounting, and Auditing Clerks                           1464
Maintenance and Repair Workers, General                                1462
Accountants and Auditors                                               1460
                                                                       ... 
Neurologists                                                              3
Surgeons                                                                  3
Ophthalmologists, Except Pediatric                                        3
Orthodontists                                                             3
Anesthesiologists                                                         2
Name: count, Length: 1047, dtype: int64

In [41]:
# Check summary statistics of the cleaned dataset
bls_df.describe()

,AREA,H_MEAN,A_MEAN,H_MEDIAN,A_MEDIAN,H_PCT75,A_PCT75,H_PCT90,A_PCT90
count,3.761160e+05,376116.000000,376116.000000,376116.000000,376116.000000,376116.000000,376116.000000,376116.000000,376116.000000
mean,3.832711e+05,30.373118,63175.027438,28.598701,59484.308618,35.265190,73350.643339,42.904900,89241.215157
std,1.142516e+06,13.357730,27784.077734,12.755897,26532.271666,16.450799,34217.675515,20.988672,43656.423537
min,1.000000e+00,8.050000,16750.000000,7.250000,15080.000000,8.010000,16670.000000,8.150000,16950.000000
25%,9.900000e+01,20.430000,42490.000000,18.960000,39450.000000,22.790000,47410.000000,27.350000,56900.000000
50%,9.900000e+01,26.680000,55500.000000,24.690000,51360.000000,30.520000,63480.000000,37.160000,77300.000000
75%,3.746000e+04,37.220000,77410.000000,35.330000,73480.000000,44.100000,91720.000000,53.510000,111310.000000
max,7.800001e+06,158.410000,329490.000000,114.320000,237780.000000,114.860000,238900.000000,114.990000,239180.000000


In [ ]:
# Group by AREA 
area_summary = bls_df.groupby('AREA')
# Display summary statistics for each area
area_summary.head(3)

,AREA,AREA_TITLE,NAICS,NAICS_TITLE,OCC_CODE,OCC_TITLE,TOT_EMP,PRIM_STATE,H_MEAN,A_MEAN,H_MEDIAN,A_MEDIAN,H_PCT75,A_PCT75,H_PCT90,A_PCT90
0,99,U.S.,000000,Cross-industry,00-0000,All Occupations,151853870,US,31.48,65470.0,23.11,48060.0,37.01,76980.0,58.40,121470.0
1,99,U.S.,000000,Cross-industry,11-0000,Management Occupations,10495770,US,66.23,137750.0,56.19,116880.0,81.29,169090.0,111.36,231620.0
5,99,U.S.,000000,Cross-industry,11-1020,General and Operations Managers,3507810,US,62.18,129330.0,48.69,101280.0,77.06,160290.0,111.59,232110.0
6,99,U.S.,000000,Cross-industry,11-1021,General and Operations Managers,3507810,US,62.18,129330.0,48.69,101280.0,77.06,160290.0,111.59,232110.0
17,99,U.S.,000000,Cross-industry,11-2033,Fundraising Managers,31810,US,64.24,133620.0,57.31,119200.0,78.20,162660.0,102.27,212730.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365983,3000006,East-Central Montana nonmetropolitan area,000000,Cross-industry,11-3021,Computer and Information Systems Managers,50,MT,61.65,128240.0,51.31,106730.0,65.90,137070.0,84.13,174990.0
366036,5100003,Northeast Virginia nonmetropolitan area,000000,Cross-industry,11-3021,Computer and Information Systems Managers,90,VA,76.70,159540.0,74.59,155150.0,94.61,196780.0,111.53,231990.0
366090,2200002,Central Louisiana nonmetropolitan area,000000,Cross-industry,11-3031,Financial Managers,210,LA,51.79,107730.0,46.62,96960.0,63.10,131260.0,79.88,166150.0
366091,2200003,Northeast Louisiana nonmetropolitan area,000000,Cross-industry,11-3031,Financial Managers,210,LA,56.71,117950.0,46.64,97010.0,64.67,134510.0,95.61,198860.0


In [ ]:
# filter dataset to include only rows where the annual mean wage (A_MEAN) or hourly wage (H_MEAN) exceeds the equivalent of $100K per year.
filtred_bls_df = bls_df[(bls_df['A_MEAN'] >= 100000) | (bls_df['H_MEAN'] >= 48.08)]
filtred_bls_df.tail()

,AREA,AREA_TITLE,NAICS_TITLE,OCC_CODE,OCC_TITLE,TOT_EMP,PRIM_STATE,H_MEAN,A_MEAN,H_MEDIAN,A_MEDIAN,H_PCT75,A_PCT75,H_PCT90,A_PCT90
1,99,U.S.,Cross-industry,110000,Management Occupations,10495770,US,66.23,137750.0,56.19,116880.0,81.29,169090.0,111.36,231620.0
5,99,U.S.,Cross-industry,111020,General and Operations Managers,3507810,US,62.18,129330.0,48.69,101280.0,77.06,160290.0,111.59,232110.0
6,99,U.S.,Cross-industry,111021,General and Operations Managers,3507810,US,62.18,129330.0,48.69,101280.0,77.06,160290.0,111.59,232110.0
17,99,U.S.,Cross-industry,112033,Fundraising Managers,31810,US,64.24,133620.0,57.31,119200.0,78.20,162660.0,102.27,212730.0
19,99,U.S.,Cross-industry,113010,Administrative Services and Facilities Managers,373920,US,56.56,117650.0,50.44,104900.0,66.45,138220.0,86.41,179730.0


In [98]:
area_title = filtred_bls_df['AREA_TITLE'].value_counts()
area= filtred_bls_df['AREA'].unique()
industry_name = filtred_bls_df['NAICS_TITLE'].unique()
state= filtred_bls_df['PRIM_STATE'].unique()
# print(area)

In [69]:
# Verify Consistency: Ensure that all columns have consistent formatting and correct data types.
filtred_bls_df.dtypes

AREA             int64
AREA_TITLE      object
NAICS_TITLE     object
OCC_CODE        object
OCC_TITLE       object
TOT_EMP         object
PRIM_STATE      object
H_MEAN         float64
A_MEAN         float64
H_MEDIAN       float64
A_MEDIAN       float64
H_PCT75        float64
A_PCT75        float64
H_PCT90        float64
A_PCT90        float64
dtype: object

In [48]:
 # convert the total employment  to a numeric to ensure consistency
filtred_bls_df['TOT_EMP'] = pd.to_numeric(filtred_bls_df['TOT_EMP'], errors='coerce')

# recheck for a null values
filtred_bls_df['TOT_EMP'].isna().sum()
filtred_bls_df=filtred_bls_df.dropna(how='any')



C:\Users\marwa\AppData\Local\Temp\ipykernel_10028\3873120035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtred_bls_df['TOT_EMP'] = pd.to_numeric(filtred_bls_df['TOT_EMP'], errors='coerce')


In [ ]:
# Ensure that AREA_TITLE, , NAICS_TITLE, OCC_CODE, and OCC_TITLE have consistent formatting (e.g., no
#  leading/trailing spaces, all uppercase/lowercase where necessary)

#str.strip to Remove leading/trailing spaces and str.title to standardize case
filtred_bls_df['AREA_TITLE'] = filtred_bls_df['AREA_TITLE'].str.strip().str.title()#strip spaces and standardize text formatting 
filtred_bls_df['OCC_TITLE'] = filtred_bls_df['OCC_TITLE'].str.strip().str.title()
filtred_bls_df['NAICS_TITLE'] = filtred_bls_df['NAICS_TITLE'].str.strip()


filtred_bls_df.dtypes

AREA             int64
AREA_TITLE      object
NAICS           object
NAICS_TITLE     object
OCC_CODE        object
OCC_TITLE       object
TOT_EMP        float64
PRIM_STATE      object
H_MEAN         float64
A_MEAN         float64
H_MEDIAN       float64
A_MEDIAN       float64
H_PCT75        float64
A_PCT75        float64
H_PCT90        float64
A_PCT90        float64
dtype: object

### Load and Clean the second data

In [71]:
# oews_data = pd.read_csv("./Ressources/educational_attainment.csv",delimiter=';')
# oews_data.head()
# oews_data.columns
Educ_data = pd.read_csv("./Ressources/usa_00006.csv",delimiter=',')
# Educ_data.columns
Educ_data.head()

,REGION,STATEICP,SEX,AGE,RACE,RACED,CITIZEN,YRIMMIG,YRSUSA1,LANGUAGE,...,QIND,QOCC,QINCBUS,QINCINVS,QINCOTHE,QINCRETI,QINCSS,QINCTOT,QINCWAGE,QINCWELF
0,32,41,1,20,1,100,0,0,0,1,...,4,4,4,4,4,4,4,4,4,4
1,32,41,1,25,1,100,0,0,0,12,...,0,0,0,0,0,0,0,0,0,0
2,32,41,1,19,1,100,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,32,41,2,18,1,100,0,0,0,1,...,4,4,4,4,4,4,4,4,4,4
4,32,41,2,39,2,200,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [72]:
Educ_data.columns
# Educ_data.reset_index()

Index(['REGION', 'STATEICP', 'SEX', 'AGE', 'RACE', 'RACED', 'CITIZEN',
       'YRIMMIG', 'YRSUSA1', 'LANGUAGE', 'LANGUAGED', 'EDUC', 'EDUCD',
       'EMPSTAT', 'EMPSTATD', 'LABFORCE', 'OCC', 'OCCSOC', 'IND', 'INDNAICS',
       'INCTOT', 'INCWAGE', 'QAGE', 'QSEX', 'QCITIZEN', 'QLANGUAG', 'QRACE',
       'QEDUC', 'QEMPSTAT', 'QIND', 'QOCC', 'QINCBUS', 'QINCINVS', 'QINCOTHE',
       'QINCRETI', 'QINCSS', 'QINCTOT', 'QINCWAGE', 'QINCWELF'],
      dtype='object')

In [ ]:
Educ_data.isnull().sum()

In [99]:
unique_values_2 = Educ_data.apply(lambda x: x.unique())
# unique_values_2

In [85]:
# filtre the data  to only keep the relevant columns
relevant_c = ['REGION','STATEICP' ,'OCCSOC', 'INCTOT', 'INCWAGE', 'EDUC', 'EDUCD', 'SEX', 'AGE']
educational_attainment_df=Educ_data[relevant_c]
educational_attainment_df.head()

,REGION,STATEICP,OCCSOC,INCTOT,INCWAGE,EDUC,EDUCD,SEX,AGE
0,32,41,352010,19874,19874,6,65,1,20
1,32,41,131111,6313,6313,10,101,1,25
2,32,41,533030,5962,5962,6,65,1,19
3,32,41,412010,5845,5845,6,65,2,18
4,32,41,537062,7014,7014,6,64,2,39


In [80]:
# filtred_owes_df=filtred_owes_df.duplicated()
educational_attainment_df.drop_duplicates(inplace=True)

C:\Users\marwa\AppData\Local\Temp\ipykernel_10028\1797593550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  educational_attainment_df.drop_duplicates(inplace=True)


In [86]:
# chck the data dtypes 
educational_attainment_df.dtypes

REGION       int64
STATEICP     int64
OCCSOC      object
INCTOT       int64
INCWAGE      int64
EDUC         int64
EDUCD        int64
SEX          int64
AGE          int64
dtype: object

In [89]:
# rename  the  occupation code to ensure consistency
educational_attainment_df= educational_attainment_df.rename(columns={'OCCSOC':'OCC_CODE'})

In [93]:
# educational_attainment_df.head()
# Filter for individuals earning $100K or more
filtred_EDU_df = educational_attainment_df[educational_attainment_df['INCTOT'] >= 100000]
filtred_EDU_df.head()

,REGION,STATEICP,OCC_CODE,INCTOT,INCWAGE,EDUC,EDUCD,SEX,AGE
58,32,41,15124X,128597,128597,6,65,1,61
60,32,41,151252,111084,111061,10,101,1,36
62,32,41,119151,459557,457102,11,114,1,66
67,32,41,192099,133273,121582,11,116,1,46
70,32,41,112022,457102,457102,11,114,1,51


In [ ]:
# Remove any non-numeric characters and ensure all codes are exactly 6 digits by padding with leading zeros
filtred_EDU_df.loc[:, 'OCC_CODE'] = filtred_EDU_df['OCC_CODE'].str.extract('(\d+)', expand=False).str.zfill(6)

In [96]:
display(filtred_EDU_df.head(2))
display(filtred_EDU_df.tail(2))

,REGION,STATEICP,OCC_CODE,INCTOT,INCWAGE,EDUC,EDUCD,SEX,AGE
58,32,41,015124,128597,128597,6,65,1,61
60,32,41,151252,111084,111061,10,101,1,36


,REGION,STATEICP,OCC_CODE,INCTOT,INCWAGE,EDUC,EDUCD,SEX,AGE
3649643,41,68,001191,180000,180000,11,114,1,62
3649646,41,68,002310,434300,422000,11,115,1,62


In [100]:
# replace all occurrences of 1 with "Male" and 2 with "Female" in the SEX column.#
filtred_EDU_df.loc[:,'SEX'] = filtred_EDU_df['SEX'].replace({1: 'Male', 2: 'Female'})


In [101]:
filtred_EDU_df.head(2)

,REGION,STATEICP,OCC_CODE,INCTOT,INCWAGE,EDUC,EDUCD,SEX,AGE
58,32,41,015124,128597,128597,6,65,Male,61
60,32,41,151252,111084,111061,10,101,Male,36


In [ ]:
# combined_df = pd.merge(filtred_bls_df,filtred_owes_df,on=['OCC_CODE', how= 'inner'])